In [2]:
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Pillow
from PIL import Image
# Torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms

In [3]:
size = (150, 150)
groups = ['train', 'test', 'val']
classes = {0: 'normal', 1: 'non-covid', 2:"covid"}

dataset_numbers = {'train_normal': 1340,\
                   'train_non-covid': 2529,\
                   'train_covid':1344,\
                   'val_normal': 7,\
                   'val_non-covid': 7,\
                   'val_covid': 8,\
                   'test_normal': 233,\
                   'test_non-covid': 241,\
                   'test_covid': 138}
print(dataset_numbers)

{'train_normal': 1340, 'train_non-covid': 2529, 'train_covid': 1344, 'val_normal': 7, 'val_non-covid': 7, 'val_covid': 8, 'test_normal': 233, 'test_non-covid': 241, 'test_covid': 138}


In [ ]:
dataset_paths = {'train_normal': './dataset/train/normal/',\
                 'train_non-covid': './dataset/train/infected/non-covid/',\
                 'train_covid': './dataset/train/infected/covid/',\
                 'val_normal': './dataset/val/normal/',\
                 'val_non-covid': './dataset/val/infected/non-covid/',\
                 'val_covid': './dataset/val/infected/covid/',\
                 'test_normal': './dataset/test/normal/',\
                 'test_non-covid': './dataset/test/infected/non-covid/',\
                 'test_covid': './dataset/test/infected/covid/'}
print(dataset_paths)